In [18]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [19]:
# Data preprocessing
data_text = """
Juana. esclava Domingo veinte y dos de y nueve yo Thomas de Orvera baptize, y pusse santos oleos a Juana de nacion Mina esclava de Juan Joseph de Justis fueron sus P.P. Joseph Salcedo y Ana de Santiago su mugger, y lo firmé. Thomas de Orvera Paula. esclava Juebes veinte y tres de febrero de mil setecientos. y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos15 oleos á Paula h. l.16 de Juan Joseph, y Maria Josepha esclavos del Capitán Don Luis Hurtado de Mendoza fue su padrino Bartholome Rixo, y lo firmé. Thomas de Orvera Maria esclava Miercoles primero de febrero de mil sietectos y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos oleos á Maria h. l. de Juan, y Josepha esclavos del Capitán Antonio Benites fue su Madrina Ysabel Mendez, y lo firmé. Thomas de Orvera Bernardo esclavo Domingo nueve de Abril de mil setecientos y diez y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Bernardo negro adulto de nacion Carabalí esclavo de Don Juan Joseph de Justis fue su Padre. Andres de Morales, y lo firmé. Thomas de Orvera Francisco esclavo Abril de mil setecientos y diez, y nueve o Thmas de vera bapizé, y pusse los s.tos oleo a Francisco negro adulto de nacion temo esclavo de Don Ju Joseph de Justis fue su Padre. Pedro Suares, y lo firmé. Thomas de Orvera Antonio esclavo Domingo nueve de Abril de mil setecientos y diez y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Antonio negro adulto de nacion Carabalí esclavo de Don Juan Joseph de Justis fue su Padre. Joseph de Soto, y lo firmé. Thomas de Orvera Antonia esclava Domingo nueve de Abril de mil setecientos y diez, y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Antonia negra adulta de nacion Mina esclava de Don Juan Joseph de Justis fue su Padre. Joseph Salcedo, y lo firmé. Thomas de Orvera Maria Luisa esclava Domingo nueve de Abril de mil setecientos y diez, y nueve Yo Thomas de Orvera baptize, y puse los santos oleos á Maria Luisa de nacion Lucumí negra adulta esclava de Don Juan Joseph de Justis fue su Padre. Jacinto de Castro, y lo firmé.
"""
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    long_words=[]
    for i in newString.split():
        if len(i)>3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

data_new = text_cleaner(data_text)

In [20]:
#Preprocessing Part 2
def create_seq(text, length = 30):
    sequences = list()
    for i in range(length, len(text)):
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

sequences = create_seq(data_new)

Total Sequences: 1523


In [21]:
# Create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences

sequences = encode_seq(sequences)

In [22]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
# Prepare the dataset
vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [23]:
# Define the model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer=Adam(lr=0.01))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 30, 50)            1200      
                                                                 
 gru_2 (GRU)                 (None, 150)               90900     
                                                                 
 dense_2 (Dense)             (None, 24)                3624      
                                                                 
Total params: 95,724
Trainable params: 95,724
Non-trainable params: 0
_________________________________________________________________


None


In [24]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5),
             ModelCheckpoint('model.h5', save_best_only=True, 
             save_weights_only=False, monitor='val_loss')]

# Fit the model
history = model.fit(X_tr, y_tr, epochs=10, batch_size=256, #Manually set to 10 to make it run faster
                    verbose=1, callbacks=callbacks, validation_data=(X_val, y_val))

Epoch 1/10
6/6 [==============================] - 7s 458ms/step - loss: 3.1645 - acc: 0.1307 - val_loss: 3.1409 - val_acc: 0.1373
Epoch 2/10
6/6 [==============================] - 2s 285ms/step - loss: 3.1017 - acc: 0.1679 - val_loss: 3.0510 - val_acc: 0.1307
Epoch 3/10
6/6 [==============================] - 2s 289ms/step - loss: 2.9442 - acc: 0.1460 - val_loss: 2.9831 - val_acc: 0.1307
Epoch 4/10
6/6 [==============================] - 2s 283ms/step - loss: 2.8366 - acc: 0.1460 - val_loss: 2.9197 - val_acc: 0.1307
Epoch 5/10
6/6 [==============================] - 2s 282ms/step - loss: 2.8210 - acc: 0.1599 - val_loss: 2.9458 - val_acc: 0.1634
Epoch 6/10
6/6 [==============================] - 2s 347ms/step - loss: 2.8035 - acc: 0.1606 - val_loss: 2.8998 - val_acc: 0.1307
Epoch 7/10
6/6 [==============================] - 3s 451ms/step - loss: 2.7846 - acc: 0.1460 - val_loss: 2.8842 - val_acc: 0.1307
Epoch 8/10
6/6 [==============================] - 2s 304ms/step - loss: 2.7680 - acc: 0.15

In [25]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    import heapq
    in_text = seed_text
    predictions = []
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre').squeeze()
        # predict character
        pred = model.predict(np.array([encoded]), verbose=0)
        # applying softmax to convert output into probabilities
        probas = np.exp(pred) / np.sum(np.exp(pred))
        # getting top 3 predictions
        top_3 = heapq.nlargest(3, zip(probas[0], list(range(len(probas[0])))))
        # reverse map integer to character for each prediction and store them
        for score, idx in top_3:
            out_char = ''
            for char, index in mapping.items():
                if index == idx:
                    out_char = char
                    break
            # store the prediction information
            predictions.append({'score': float(score), 'token': idx, 'token_str': out_char, 'sequence': in_text + out_char})
        # continue generating based on the top prediction
        in_text += predictions[-1]['token_str']
    return predictions


In [26]:
print(generate_seq(model, mapping, 30, 'juana esclava domingo', 100))

[{'score': 0.0499182753264904, 'token': 0, 'token_str': ' ', 'sequence': 'juana esclava domingo '}, {'score': 0.043836940079927444, 'token': 5, 'token_str': 'e', 'sequence': 'juana esclava domingoe'}, {'score': 0.043575968593358994, 'token': 17, 'token_str': 's', 'sequence': 'juana esclava domingos'}, {'score': 0.0505598708987236, 'token': 0, 'token_str': ' ', 'sequence': 'juana esclava domingos '}, {'score': 0.04469212889671326, 'token': 5, 'token_str': 'e', 'sequence': 'juana esclava domingose'}, {'score': 0.044177815318107605, 'token': 1, 'token_str': 'a', 'sequence': 'juana esclava domingosa'}, {'score': 0.05145537853240967, 'token': 0, 'token_str': ' ', 'sequence': 'juana esclava domingosa '}, {'score': 0.043647997081279755, 'token': 5, 'token_str': 'e', 'sequence': 'juana esclava domingosae'}, {'score': 0.04326777905225754, 'token': 17, 'token_str': 's', 'sequence': 'juana esclava domingosas'}, {'score': 0.051236290484666824, 'token': 0, 'token_str': ' ', 'sequence': 'juana escla